In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qs_mps.applications.Z2.paulis import PauliTensor, PauliTensorSum, commutator
from sympy.physics.paulialgebra import Pauli
from qs_mps.applications.Z2.geometry import FrancescosDualCylinder
from sympy.core.numbers import One
import numpy as np

def Z2_hamiltonian_z_terms(dual_lattice: FrancescosDualCylinder, g: float):
    if g == 0:
        return PauliTensorSum()
    
    # Couplings between dual physical degrees of freedom
    each_interaction_qubits = np.zeros((dual_lattice.num_edges + dual_lattice.py, 2))
    for i, edge in enumerate(dual_lattice.edges):
        edge_nodes = sorted(dual_lattice.nodes_connected_to_edge(edge))
        each_interaction_qubits[i] = dual_lattice.coords_to_index(edge_nodes)

    # Couplings to topological degree of freedom
    for i in range(dual_lattice.py):
        each_interaction_qubits[-dual_lattice.py + i] = [dual_lattice.num_nodes - dual_lattice.py + i, dual_lattice.num_nodes]
    
    z_terms = [PauliTensor([Pauli(3) if j in this_int_qubits else One for j in range(dual_lattice.num_nodes + 1)], coefficient=-g) for this_int_qubits in each_interaction_qubits]
    return PauliTensorSum(*z_terms)

def Z2_hamiltonian_x_terms(dual_lattice: FrancescosDualCylinder, g: float):
    if g == 0:
        return PauliTensorSum()
    
    x_terms = [PauliTensor([Pauli(1) if j == this_index else One for j in range(dual_lattice.num_nodes)] + [One], coefficient=-1/g) for this_index in range(dual_lattice.num_nodes)]
    return PauliTensorSum(*x_terms)

def Z2_trotter_dt(dual_lattice, g, error):
    z_terms = Z2_hamiltonian_z_terms(dual_lattice, g)
    x_terms = Z2_hamiltonian_x_terms(dual_lattice, g)
    z_z_x_commutator_norm = commutator(z_terms, commutator(z_terms, x_terms)).norm_upper_bound()
    x_x_z_commutator_norm = commutator(x_terms, commutator(x_terms, z_terms)).norm_upper_bound()
    return (error/(z_z_x_commutator_norm/12 + x_x_z_commutator_norm/24))**(1/3)

In [28]:
dual_lattice = FrancescosDualCylinder(30, 5)
print(dual_lattice.nodes)
print(dual_lattice.edges)
print(dual_lattice.coords)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (11, 0), (11, 1), (11, 2), (11, 3), (11, 4), (12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (13, 0), (13, 1), (13, 2), (13, 3), (13, 4), (14, 0), (14, 1), (14, 2), (14, 3), (14, 4), (15, 0), (15, 1), (15, 2), (15, 3), (15, 4), (16, 0), (16, 1), (16, 2), (16, 3), (16, 4), (17, 0), (17, 1), (17, 2), (17, 3), (17, 4), (18, 0), (18, 1), (18, 2), (18, 3), (18, 4), (19, 0), (19, 1), (19, 2), (19, 3), (19, 4), (20, 0), (20, 1), (20, 2), (20, 3), (20, 4), (21, 0), (21, 1), (21, 2), (21, 3), (21, 4), (22, 0), (22, 1), (22, 2), (22, 3), (22, 4), (23, 0), (23, 

In [25]:
Z2_hamiltonian_z_terms(dual_lattice, 2)

-2ZZIIIII - 2ZIZIIII - 2IZIZIII - 2IIZZIII - 2IIZIZII - 2IIIZIZI - 2IIIIZZI - 2IIIIZIZ - 2IIIIIZZ

In [26]:
Z2_hamiltonian_x_terms(dual_lattice, 2)

-0.5XIIIIII - 0.5IXIIIII - 0.5IIXIIII - 0.5IIIXIII - 0.5IIIIXII - 0.5IIIIIXI

In [29]:
from sympy import symbols

g = symbols("g")
epsilon = symbols(r"\varepsilon")

Z2_trotter_dt(dual_lattice, g, epsilon)

(\varepsilon/(2365*Abs(g)/3 + 395/(2*Abs(g))))**0.333333333333333

In [32]:
g = 0.75
epsilon = 1e-2

(epsilon/(2365*g/3 + 395/(2*g)))**(1/3)

0.022702927405028807